# Betfair - Bfexplorer BOT SDK

More information about betfair trading software:

http://bfexplorer.net/Products/Bfexplorer

Bfexplorer BOT SDK:

https://github.com/StefanBelo/Bfexplorer-BOT-SDK

# Setup

1)	Install bfexplorer software: http://bfexplorer.net/Products/FreeTrial

2)	Open bfexplorer app, and login with your betfair credentials to activate your betfair api access.

To use bfexplorer assemblies we must reference them:

In [34]:
#I "C:\Program Files (x86)\BeloSoft\Bfexplorer"

#r "BeloSoft.Data.dll"
#r "BeloSoft.Betfair.API.dll"
#r "BeloSoft.Bfexplorer.API.dll"

#r "BeloSoft.Bfexplorer.Domain.dll"
#r "BeloSoft.Bfexplorer.Service.dll"
#r "BeloSoft.Bfexplorer.Service.Core.dll"    

In [35]:
open BeloSoft.Data
open BeloSoft.Bfexplorer.Domain
open BeloSoft.Bfexplorer.Service

In [36]:
let bfexplorerService = BfexplorerService()

In [37]:
// Your betfair credentials should be added to the script file named: BetfairCredentials.fsx

(*
module MyData.Credentials
    
[<Literal>]
let Username = "your user name"

[<Literal>]
let Password = "your password"
*)

#load "BetfairCredentials.fsx"
    
open MyData

// Or just type your credentials directly to Login function:

// let loginResult = bfexplorerService.Login(“username”, “password”) |> Async.RunSynchronously

In [38]:
let loginResult = bfexplorerService.Login(Credentials.Username, Credentials.Password) |> Async.RunSynchronously

In [39]:
printfn "%s" <|
    if loginResult.IsSuccessResult
    then
        "Login to betfair has been successful."
    else
        let _isAuthorized, _isActiveSubscriber, errorMessage = loginResult.FailureResult
        
        sprintf "Failed to login: %s" errorMessage

Login to betfair has been successful.


Get market catalogues for win horse racing markets in GB and IE.

In [40]:
let filter = [ BetEventTypeIds [| 7 |]; MarketTypeCodes [| "WIN" |]; Countries [| "GB"; "IE" |] ]

In [41]:
let marketCataloguesResult = bfexplorerService.GetMarketCatalogues(filter, 1000) |> Async.RunSynchronously

In [9]:
printfn "%s" <|
    match marketCataloguesResult with
    | DataResult.Success marketCatalogues ->

        marketCatalogues
        |> Seq.map (fun marketCatalogue -> sprintf "%A" marketCatalogue.MarketInfo)
        |> String.concat "\n"

    | DataResult.Failure errorMessage -> sprintf "Failed to GetMarketCatalogues: %s" errorMessage

Catt 1st Jan - 12:55 3m1f Beg Chs
Tram 1st Jan - 13:00 2m Mdn Hrd
Fake 1st Jan - 13:05 2m4f Hcap Hrd
Sthl 1st Jan - 13:10 1m Hcap
Chelt 1st Jan - 13:15 2m4f Listed Nov Hrd
Fairy 1st Jan - 13:20 2m4f Mdn Hrd
Catt 1st Jan - 13:25 2m3f Nov Hrd
Muss 1st Jan - 13:30 1m7f Juv Hrd
Tram 1st Jan - 13:35 2m Hcap Hrd
Fake 1st Jan - 13:35 3m5f Hcap Chs
Extr 1st Jan - 13:40 2m3f Hcap Chs
Sthl 1st Jan - 13:45 5f Nov Stks
Chelt 1st Jan - 13:50 3m2f Hcap Chs
Fairy 1st Jan - 13:55 2m2f Nov Hrd
Catt 1st Jan - 14:00 1m7f Hcap Hrd
Muss 1st Jan - 14:05 3m Hcap Hrd
Tram 1st Jan - 14:10 2m5f Mdn Hrd
Fake 1st Jan - 14:10 2m4f Hcap Hrd
Extr 1st Jan - 14:15 2m3f Nov Hcap Chs
Sthl 1st Jan - 14:20 5f Hcap
Chelt 1st Jan - 14:25 2m4f Grd2 Nov Chs
Fairy 1st Jan - 14:30 2m Hcap Hrd
Catt 1st Jan - 14:35 1m7f Hcap Hrd
Muss 1st Jan - 14:40 2m4f Hcap Chs
Fake 1st Jan - 14:45 2m7f Hcap Hrd
Tram 1st Jan - 14:45 2m5f Hcap Hrd
Extr 1st Jan - 14:50 2m2f Hcap Hrd
Sthl 1st Jan - 14:55 7f Hcap
Chelt 1st Jan - 15:00 2m4f Grd3 Hca

Take the first market and load selections data:

In [42]:
let marketCatalogue = marketCataloguesResult.SuccessResult |> Seq.head

let marketResult = bfexplorerService.GetMarket(marketCatalogue.MarketInfo) |> Async.RunSynchronously


In [43]:
let toString (market : Market) =
    sprintf "%s\n%s" market.MarketFullName (
        market.Selections
        |> Seq.filter (fun selection -> selection.Status = SelectionStatus.Active)
        |> Seq.map (fun selection -> sprintf "\t%s: %.2f | %.2f" selection.Name selection.LastPriceTraded selection.TotalMatched)
        |> String.concat "\n"
    )


In [44]:
printfn "%s" <|
    match marketResult with
    | DataResult.Success market -> toString market
    | DataResult.Failure errorMessage -> sprintf "Failed to GetMarket: %s" errorMessage    

Extr 1st Jan - 15:25 3m Hcap Chs
	Captain Drake: 1.01 | 140740.12
	Western Climate: 1000.00 | 44808.68
	Know The Score: 1000.00 | 59417.62
	Snuff Box: 1000.00 | 24304.22
	The Kings Writ: 1000.00 | 40149.64
	Dancing Shadow: 1000.00 | 16637.15
	Barbrook Star: 1000.00 | 17679.45
	Muffins For Tea: 1000.00 | 20683.09
	Mr Caffrey: 1000.00 | 16197.79
	Coopers Square: 1000.00 | 8348.64


Update the market data:

In [28]:
let market = marketResult.SuccessResult

let showMarketData() =
    market |> toString |> printfn "%s"

In [31]:
let updateMarketResult = bfexplorerService.UpdateMarketBaseData(market) |> Async.RunSynchronously

if updateMarketResult.IsSuccessResult
then
    showMarketData()

Catt 1st Jan - 12:55 3m1f Beg Chs
	Deise Aba: 1.72 | 25620.72
	Jack Devine: 2.38 | 14644.39


All done, we can logout now:

In [32]:
let resultLogout = bfexplorerService.Logout() |> Async.RunSynchronously 

In [33]:
printfn "%A" resultLogout.IsSuccessResult

true
